# Basic Testing

**TBD: in `DSA_utils.py` need to add more functions from `Utils/TCGA_survival_utils.py` to it.**

**TBD: NEED TO DECIDE IF TO change cancer values in mutation data to be without the TCGA_ prefix, so that it will be the same as in the clinical data ????**. This requires chnages to:
- `TCGA_interface.py` - when returning the mutation dataframe
- `DSA_utils.py` in `process_gene_muts_survival_tests`, in the line  `cancer_name_in_clinical = gdcu.TCGA_proj_full2name(cancer)`

In [1]:
import pathlib
import pandas as pd
import mysys_init as si
import GDC_dataset_feb2021_utils as gdcu

from Utils import TCGA_interface as TCGAi
from Utils import DSA_utils as ut
from Utils.interface import DSA_Mutation_ID,DSA_data,DSA_compute

report_path = pathlib.Path(si.ROOT_path) / 'Explore_kaplan_meier' / 'Reports'

# Parameters
Should move to a config file.

In [2]:
# TCGA clinical file
TCGA_clinical_data_file: pathlib.Path = pathlib.Path(si.ROOT_path) / 'Data' / 'GDC' / 'Clinical' / 'df_p_all.pkl'

# TCGA main mutation folder
TCGA_mutation_data_folder: pathlib.Path = pathlib.Path(si.ROOT_path) / 'Data' / 'GDC' / 'Mutations' / 'data' / 'filtered_feb_2021' / 'AllGenes' if si.MACHINE == 'MAC' else pathlib.Path(gdcu.Dataset_main_folder)

min_drug_prevalence: int = 20  #50  #100
TCGA_df_clinical_drugs_dict_file_name: pathlib.Path = report_path / 'TCGA_clinical_preprocess' / f'df_clinical_drugs_dict_{min_drug_prevalence}.pkl'

# TEST 1

In [9]:
def my_app(Mutid: DSA_Mutation_ID, Dsa_d: DSA_data, Dsa_c: DSA_compute) -> None:
    """Dummy application."""
    print("Mutation data:")
    print(f"{Mutid.mut_id_cols}")

    print("\nDSA data:")
    print(f"Clinical data")
    display(Dsa_d.get_clinical_data())
    print(f"patients and cancer columns are {Dsa_d.patient_ID_col} and {Dsa_d.cancer_col}.")
    print(f"{Dsa_d.cancer_col} values are: {Dsa_d.get_clinical_data()[Dsa_d.cancer_col].unique().tolist()}")

    print(f"\nPatients per cancer:")
    a = [(cnr, df_c.shape[0]) for cnr, df_c in Dsa_d.get_clinical_data().groupby(by=Dsa_d.cancer_col)]
    df_cancer_prevalence = pd.DataFrame({'Cancer': [x[0] for x in a], 'Num_patients': [x[1] for x in a]}).sort_values('Num_patients', ascending=False).reset_index(drop=True)
    df_cancer_prevalence.index += 1
    df_cancer_prevalence['Cancer_name'] = df_cancer_prevalence.apply(lambda x: gdcu.Cancer_abbrv2name[f"TCGA_{x['Cancer']}"], axis=1)
    df_cancer_prevalence = df_cancer_prevalence[['Cancer', 'Cancer_name', 'Num_patients']]
    display(df_cancer_prevalence)

    print("\nPatients per therapy:")
    df_clinical_therapies_dict = Dsa_d.df_clinical_therapies_dict
    df_therapy_prevalence = pd.DataFrame({'Therapy': list(df_clinical_therapies_dict.keys()), 'Num_patients': [x.shape[0] for x in df_clinical_therapies_dict.values()]}).sort_values('Num_patients', ascending=False).reset_index(drop=True)
    df_therapy_prevalence.index += 1
    display(df_therapy_prevalence)

    gene: str = 'TPX2'
    if (df_gene := Dsa_d.get_gene_mutations(gene)) is None:
        raise ValueError(f"No mutation data for {gene=}.")
    print(f"{gene} mutation data:")
    display(df_gene)
    print(f"{Dsa_d.cancer_col} values are: {df_gene[Dsa_d.cancer_col].unique().tolist()}")
    print("NEED TO CHANGE TCGA_X to X ??????????????????????? (OR change in the clinical data X to TCGA_X ?????????)")

    # filtered mutations
    filter_data: dict = {
        'mut_id_col_to_add': 'mut_id',
        'filter_silent': True,
        'filter_TCGA_projects': True,
        'mutation_min_affected_number_of_patients': 10
    }
    gn_muts, df1, df2 = Dsa_d.get_filtered_gene_mutations(gene, filter_data)
    print(f"Filter data of {gene=} (contains {len(gn_muts)} mutations)")
    display(df1)


    mid: str = 'TPX2:chr20:Splice_Region:31770476:G:T'
    df_mid = Mutid.get_df_mut(df_gene, mid)
    display(df_mid)

    print("\nCompute data:")
    print(Dsa_c.P_val_threshold)


In [10]:
# instantiate TCGA data
Mut = TCGAi.TCGA_Mutation_ID()
Dsa_data = TCGAi.TCGA_data(TCGA_mutation_data_folder, TCGA_clinical_data_file, TCGA_df_clinical_drugs_dict_file_name)
Dsa_compute = TCGAi.TCGA_compute()

# run the "app"
my_app(Mut, Dsa_data, Dsa_compute)


Mutation data:
['Gene_name', 'Chromosome', 'Variant_Classification', 'Start_Position', 'Reference_Allele', 'Tumor_Seq_Allele2']

DSA data:
Clinical data


,patient_barcode,case_id,sample_barcode,sample_uuid,female,male,race,ethnicity,days_to_birth,alive,...,primary_tumor,blood_derived_normal,solid_tissue_normal,Proj_name,metastatic,recurrent_tumor,bone_marrow_normal,buccal_cell_normal,tumor,additional_metastatic
0,tcga-or-a5ld,d885b720-e813-4828-a731-b3cdcbd20687,tcga-or-a5ld-10a,7f0378a8-e904-404b-91de-a70e73bda1b4,NaN,1.0,white,not hispanic or latino,19111.0,NaN,...,1.0,1.0,NaN,ACC,NaN,NaN,NaN,NaN,NaN,NaN
1,tcga-or-a5jd,240dca72-76e0-4d57-978d-1242e6e7ed68,tcga-or-a5jd-10b,4fdc28a5-f64e-4d98-baac-befad27fb2a1,1.0,NaN,white,NaN,20989.0,3.0,...,1.0,1.0,NaN,ACC,NaN,NaN,NaN,NaN,NaN,NaN
2,tcga-or-a5jm,fb54458d-c373-46c2-841e-82663e13efaa,tcga-or-a5jm-10b,a3c80a41-bce1-4f72-8195-f1c699957c5f,1.0,NaN,white,NaN,9359.0,NaN,...,1.0,1.0,NaN,ACC,NaN,NaN,NaN,NaN,NaN,NaN
3,tcga-or-a5jz,94cdac79-2b4e-483a-8f17-3e0c9be589d0,tcga-or-a5jz-10a,a9c6d6e4-ab79-4a75-bd63-891647d82501,NaN,1.0,white,NaN,22218.0,2.0,...,1.0,1.0,NaN,ACC,NaN,NaN,NaN,NaN,NaN,NaN
4,tcga-ou-a5pi,a5be4e42-cb6c-4e67-8547-45dbd058bb95,tcga-ou-a5pi-10a,a37b1255-a17c-49e9-97f7-8525aee7438b,1.0,NaN,white,not hispanic or latino,19492.0,2.0,...,1.0,1.0,NaN,ACC,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,tcga-vd-aa8q,1d00217f-28df-4d91-aeb9-0d9a83a907e6,tcga-vd-aa8q-10a,d57d0601-a8eb-471e-9055-109caea0ecb9,NaN,1.0,NaN,NaN,26285.0,2.0,...,1.0,1.0,NaN,UVM,NaN,NaN,NaN,NaN,NaN,NaN
11137,tcga-v4-a9ej,d9ccba9b-6bf0-44d9-8097-a97ab747f0bd,tcga-v4-a9ej-10a,cef70e8e-da78-4eec-ab47-63a8d2be2cc4,NaN,1.0,white,not hispanic or latino,13958.0,3.0,...,1.0,1.0,NaN,UVM,NaN,NaN,NaN,NaN,NaN,NaN
11138,tcga-wc-a87u,4498044c-1607-403e-867e-befd7fad275a,tcga-wc-a87u-10a,929e6183-8e31-44ee-a2d9-89b3e8861e01,NaN,1.0,white,not hispanic or latino,26933.0,NaN,...,1.0,1.0,NaN,UVM,NaN,NaN,NaN,NaN,NaN,NaN
11139,tcga-v4-a9eh,5e54122f-1c4d-45cc-8842-cd1c1df0eed8,tcga-v4-a9eh-10a,9256ff7e-5674-4e61-9fdc-947669e525c7,NaN,1.0,white,hispanic or latino,19573.0,2.0,...,1.0,1.0,NaN,UVM,NaN,NaN,NaN,NaN,NaN,NaN


patients and cancer columns are case_id and Proj_name.
Proj_name values are: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']

Patients per cancer:


,Cancer,Cancer_name,Num_patients
1,BRCA,Breast invasive carcinoma,1095
2,GBM,Glioblastoma multiforme,594
3,OV,Ovarian serous cystadenocarcinoma,585
4,UCEC,Uterine Corpus Endometrial Carcinoma,546
5,KIRC,Kidney renal clear cell carcinoma,535
6,HNSC,Head and Neck squamous cell carcinoma,526
7,LUAD,Lung adenocarcinoma,521
8,LGG,Brain Lower Grade Glioma,513
9,THCA,Thyroid carcinoma,505
10,LUSC,Lung squamous cell carcinoma,502



Patients per therapy:


...

,Proj_name,Gene_name,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,...,nontcga_ExAC_AF_NFE,nontcga_ExAC_AF_OTH,nontcga_ExAC_AF_SAS,hotspot,callers,Disease_type,FMI_STATUS,FMI_GENE,FMI_TRANSCRIPT,FMI_FUNCTIONAL_EFFECT
0,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31797557,31797557,+,Intron,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
1,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31797463,31797463,+,Silent,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
2,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31766551,31766551,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
3,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31793888,31793888,+,Missense_Mutation,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
4,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31797353,31797353,+,Intron,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,TARGET_OS,TPX2,22974,NCI-MELTZER,GRCh38,chr20,31792728,31792728,+,Splice_Region,...,0.0,0.0,0.000061,N,muse,<undefined>,<undefined>,NaN,NaN,<undefined>
4182,TARGET_OS,TPX2,22974,NCI-MELTZER,GRCh38,chr20,31792732,31792732,+,Splice_Region,...,NaN,NaN,NaN,N,muse,<undefined>,<undefined>,NaN,NaN,<undefined>
4183,TARGET_WT,TPX2,22974,BCM,GRCh38,chr20,31777780,31777781,+,Intron,...,NaN,NaN,NaN,N,varscan2,<undefined>,<undefined>,NaN,NaN,<undefined>
4184,TARGET_WT,TPX2,22974,BCM,GRCh38,chr20,31770177,31770177,+,Intron,...,NaN,NaN,NaN,N,muse,<undefined>,<undefined>,NaN,NaN,<undefined>


Proj_name values are: ['TCGA_SKCM', 'TCGA_THYM', 'TCGA_BRCA', 'TCGA_GBM', 'TCGA_STAD', 'TCGA_LAML', 'TCGA_CHOL', 'TCGA_LIHC', 'TCGA_MESO', 'TCGA_OV', 'TCGA_UCEC', 'TCGA_LUAD', 'TCGA_ESCA', 'TCGA_PAAD', 'TCGA_KICH', 'TCGA_THCA', 'TCGA_READ', 'TCGA_KIRC', 'TCGA_PRAD', 'TCGA_COAD', 'TCGA_HNSC', 'TCGA_SARC', 'TCGA_UCS', 'TCGA_UVM', 'TCGA_LUSC', 'TCGA_PCPG', 'TCGA_ACC', 'TCGA_CESC', 'TCGA_DLBC', 'TCGA_KIRP', 'TCGA_TGCT', 'TCGA_BLCA', 'TCGA_LGG', 'TARGET_ALL-P2', 'TARGET_ALL-P3', 'TARGET_AML', 'TARGET_NBL', 'TARGET_OS', 'TARGET_WT']
NEED TO CHANGE TCGA_X to X ??????????????????????? (OR change in the clinical data X to TCGA_X ?????????)
Filter data of gene='TPX2' (contains 31 mutations)


,Proj_name,Gene_name,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,...,nontcga_ExAC_AF_OTH,nontcga_ExAC_AF_SAS,hotspot,callers,Disease_type,FMI_STATUS,FMI_GENE,FMI_TRANSCRIPT,FMI_FUNCTIONAL_EFFECT,mut_id
13,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31757469,31757469,+,5'UTR,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:5'UTR:31757469:G:T
35,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31757469,31757469,+,5'UTR,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:5'UTR:31757469:G:T
37,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31766455,31766462,+,Intron,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:Intron:31766455:GTGTGTGT:-
52,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31766455,31766458,+,Intron,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:Intron:31766455:GTGT:-
53,TCGA_SKCM,TPX2,22974,BI,GRCh38,chr20,31766455,31766460,+,Intron,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:Intron:31766455:GTGTGT:-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31766455,31766460,+,Intron,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:Intron:31766455:GTGTGT:-
3316,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31794305,31794305,+,Intron,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:Intron:31794305:T:G
3317,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31794512,31794512,+,Silent,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:Silent:31794512:A:G
3318,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31766339,31766339,+,Intron,...,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>,TPX2:chr20:Intron:31766339:G:A


,Proj_name,Gene_name,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,...,nontcga_ExAC_AF_NFE,nontcga_ExAC_AF_OTH,nontcga_ExAC_AF_SAS,hotspot,callers,Disease_type,FMI_STATUS,FMI_GENE,FMI_TRANSCRIPT,FMI_FUNCTIONAL_EFFECT
361,TCGA_BRCA,TPX2,22974,WUGSC,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
365,TCGA_BRCA,TPX2,22974,WUGSC,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
374,TCGA_BRCA,TPX2,22974,WUGSC,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
386,TCGA_BRCA,TPX2,22974,WUGSC,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
402,TCGA_BRCA,TPX2,22974,WUGSC,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4103,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
4114,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
4117,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>
4136,TCGA_LGG,TPX2,22974,BI,GRCh38,chr20,31770476,31770476,+,Splice_Region,...,NaN,NaN,NaN,<undefined>,<undefined>,<undefined>,<undefined>,NaN,NaN,<undefined>



Compute data:
0.05


In [6]:
# more testing

muts_ids: list[str] = [
    'EGFR:chr7:Intron:55201370:TC:-',
    'BRCA1:chr17:Intron:43090922:CA:-',
    'IDH1:chr2:Intron:208245452:A:-',
    'PIK3CA:chr3:Intron:179203524:G:C',  # note that this mutation is also listed above
    'MET:chr7:Intron:116769622:TTT:-',
    'ROS1:chr6:Intron:117310301:TAA:-',
    'PIK3CA:chr3:Intron:179224066:T:-',
    'PIK3CA:chr3:Intron:179220948:TA:-'
]

k = ut.create_gene_mutations_dict(muts_ids, Mut)
print(k)

{'EGFR': ['EGFR:chr7:Intron:55201370:TC:-'], 'BRCA1': ['BRCA1:chr17:Intron:43090922:CA:-'], 'IDH1': ['IDH1:chr2:Intron:208245452:A:-'], 'PIK3CA': ['PIK3CA:chr3:Intron:179203524:G:C', 'PIK3CA:chr3:Intron:179224066:T:-', 'PIK3CA:chr3:Intron:179220948:TA:-'], 'MET': ['MET:chr7:Intron:116769622:TTT:-'], 'ROS1': ['ROS1:chr6:Intron:117310301:TAA:-']}


In [ ]:
a = 'MCT-mCT,MCt-mCt'
a_cfg = ut.DS_test_name2config(a)
print(a_cfg)

In [ ]:
cur_therapy = 'chemotherapy'
# ==========================
cur_clinical_df = Dsa_data.df_clinical_therapies_dict[cur_therapy]
patient_list = cur_clinical_df[Dsa_data.patient_ID_col].unique().tolist()[:10]
print(patient_list)

print(cur_clinical_df.columns)
display(cur_clinical_df)

k = ut.prepare_clinical_for_KM(patient_list, cur_therapy, Dsa_data)
display(k)

# ScratchPad